<div style="background-color:#e2f0d9; padding:15px; border-radius:15px; color:#003366; font-family:Arial; font-size:18px;">

<span style="font-size:30px; font-weight:bold;">1. Analysis of the `Menu` Table Data</span><br>

Now we will understand the Menus across all restaurants.
</div>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from tabulate import tabulate
from sqlalchemy import create_engine, text

username = "root"
password = "password"
host     = "localhost"
port     = "3306"
database = "sql_jupyter_p1"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>1. max., min. and average prices across all categories</b>

In [2]:
df = pd.read_sql("""

SELECT 
    MIN(price) AS Min_price, 
    MAX(price) AS Max_price, 
    ROUND(AVG(price), 2) AS Avg_price
FROM menu 
WHERE price IS NOT NULL;

""", con=engine)

print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+-------------+-------------+-------------+
|   Min_price |   Max_price |   Avg_price |
|-------------+-------------+-------------|
|           0 |     1099.99 |       10.75 |
+-------------+-------------+-------------+


#### To know: 
 the `price range` and `average` pricing of all menu items.
#### Insight:
Most items are `cheap`, but some are `very costly`. The average price is `$10.37`.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>2. Available unique menu items and unique categories of food.</b>
</div>

In [3]:
df = pd.read_sql("""
select count(distinct(name)) as Unique_menu_items
from menu
where name is not null;
""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+---------------------+
|   Unique_menu_items |
|---------------------|
|               21589 |
+---------------------+


In [4]:
df = pd.read_sql("""

select count(distinct(category)) as Unique_categories 
from menu
where category is not null;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+---------------------+
|   Unique_categories |
|---------------------|
|                1947 |
+---------------------+


#### Why:
- To know how many `different food items` and `categories` are listed.
#### Insight:
- There are `900341` unique dishes spread across `68876` food categories.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>3. Unique menu items in each category across all restaurants.</b>
</div>

In [5]:
df = pd.read_sql("""

select category, count(distinct(name)) as item_count from menu
where name is not null
group by category
order by item_count desc
limit 10;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+----------------+--------------+
| category       |   item_count |
|----------------+--------------|
| Picked for you |         1986 |
| Sides          |          743 |
| Appetizers     |          731 |
| BEVERAGES      |          599 |
| Sandwiches     |          455 |
| Drinks         |          379 |
| Desserts       |          361 |
| Salads         |          334 |
| Mains          |          291 |
| Entrées        |          250 |
+----------------+--------------+


#### Why:
- To understand which food categories have the `highest` number of items.
#### Insight:
- `Fast food` and `beverages` dominate the menu, showing what’s most frequently offered across restaurants.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>4. Category which has the highest average item price across all restaurants.</b>
</div>

In [6]:
df = pd.read_sql("""

select category, round(avg(price)) as average_price from menu
where price is not null
group by category
order by average_price desc
limit 1;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+------------+-----------------+
| category   |   average_price |
|------------+-----------------|
| Strollers  |             401 |
+------------+-----------------+


#### To know :
- Which category has the most `expensive` items on average.

#### Insight : 
- `Strollers` category has the highest average price - `$416.0` per item.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>5. Items that are priced above the average price across all restaurants.</b>
</div>

In [ ]:
df = pd.read_sql("""

select category, name, price from menu
where price > (select avg(price) from menu
where price is not null)
order by price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

#### Why:
- To find all items that `cost` more than the `average price`.

#### Insight:
- There are about `1891077` items priced above average. Top ones are in `Curries` and `Strollers` categories.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>6. Most expensive items per category.</b>
</div>

In [ ]:
df = pd.read_sql("""

SELECT category, name AS most_expensive_item, price
FROM (
  SELECT category, name, price,
        RANK() OVER (PARTITION BY category ORDER BY price desc) AS price_rank
    FROM menu
    where price is not null and price != 0
    ) as ranked_items
where price_rank = 1
order by price desc;
  
""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

#### Why:
- To find the costliest item in each category.

#### Insight:
- Every category has some expensive items in the menu.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>7. Least expensive items per category.</b>
</div>

In [85]:
df = pd.read_sql("""

SELECT category, name AS cheapest_item, price 
FROM (
    SELECT category, name, price,
        rank() OVER (PARTITION BY category ORDER BY price ASC) AS price_rank
    FROM menu
) ranked_items
where price_rank = 1 and price != 0
order by price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,category,cheapest_item,price
0,for Clarivel &lt;3,Jason,375.00
1,Christmas and New Year Specials (Non -Vegetarian),Non Veg Party Pack ( 12 to 15 people),269.99
2,Christmas and New Year Specials (Vegetarian),Veg Party Pack (12 - 15 People),269.99
3,Dinner for 10,Shredded Duck with Pickled Vegetable,258.88
4,Dinner for 10,Shredded Chicken Salad with Sesame Oil,258.88
...,...,...,...
224711,Appertizers,Order Test 2012,0.01
224712,BBQ Necessities,"Condiments, BBQ Sauce, Peppers, etc",0.01
224713,Drinks,Apple Juice,-2.50
224714,Fajitas,Chicken en Mole,-7.05


#### To know:
- Which item is the cheapest in every category.

#### Insight:
- Each category has some low-cost items listed in the menu.

<div style="background-color:#e2f0d9; padding:15px; border-radius:15px; color:#003366; font-family:Arial; font-size:18px;">

<span style="font-size:30px; font-weight:bold;">2. Analysis of the `Restro` Table Data</span><br>

Now we will understand the restaurant profiles and performance.
</div>

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>1. Total unique 'States' count.</b>
</div>

In [86]:
df = pd.read_sql("""

select 
    count(distinct(state)) as Unique_state_count
from restro
where state is not null;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,Unique_state_count
0,21


#### To know:
- How many unique states are there.

#### Insight:
- There are 21 unique states in the data.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>2. Total unique 'Restaurants' count.</b>
</div>

In [87]:
df = pd.read_sql("""

select 
    count(distinct(name)) as Unique_Restaurants_count
from restro
where name is not null;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,Unique_Restaurants_count
0,60487


#### To know:
- How many unique restaurants are in the data.

#### Insight:
- There are 60487 unique restaurants in total.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>3. Minimum, Maximum, Average rating of all restaurants.</b>
</div>

In [110]:
df = pd.read_sql("""

select 
    min(score) as Minimum_rating,
    max(score) as Maximum_rating,
    avg(score) as Average_rating
from restro
where score is not null;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,Minimum_rating,Maximum_rating,Average_rating
0,1.3,5.0,4.551118


#### To know:
- The rating range and average rating of all restaurants.

#### Insight:
- Restaurant ratings range from 1.3 to 5.0, with an average of 4.55.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>4. Total unique 'Positions' count.</b>
</div>

In [89]:
df = pd.read_sql("""

select 
    count(distinct(position)) as Unique_positions_count
from restro
where position is not null;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,Unique_positions_count
0,300


<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>5. Restaurant with highest rating score.</b>
</div>

In [90]:
df = pd.read_sql("""

select 
    state, name, score
from restro
where score is not null and score = 5;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,state,name,score
0,Alabama,Jeni's Splendid Ice Cream (Pepper Place),5.0
1,Alabama,Wasabi Juan's (Downtown),5.0
2,Alabama,"Honey Baked Ham (7001 Crestwood Blvd, Ste 114)",5.0
3,Alabama,Cookie Dough Magic,5.0
4,Alabama,Papa Johns (736 Montgomery Hwy),5.0
...,...,...,...
918,Texas,2 and 1 Restaurant,5.0
919,Texas,Purify Bowls &amp; Smoothies,5.0
920,Texas,Ostioneria Michoacan ( Wirt Rd ),5.0
921,Texas,Urban Value Corner Store,5.0


#### To know:
- Which restaurants have the highest rating score.

#### Insight:
- 923 restaurants have a perfect 5.0 rating across different cities.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>6. Restaurants count by State.</b>
</div>

In [91]:
df = pd.read_sql("""

select 
    state, count(name) as Restro_count
from restro
where name is not null 
group by state
order by Restro_count desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,state,Restro_count
0,Texas,30310
1,Washington,10380
2,Virginia,9264
3,Wisconsin,4311
4,Utah,3067
5,West Virginia,1379
6,Alabama,1107
7,Oregon,1030
8,Maryland,894
9,None,453


#### To know:
- How many restaurants are listed in each state.

#### Insight:
- Texas has the highest `30310` number of restaurants. `Nebraska` and `Missouri` have only one.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>7. Top 10 States by Number of restaurants that have a position between 1 and 5.</b>
</div>

In [92]:
df = pd.read_sql("""

select 
    state, 
    count(name) as restro_count
from restro
where position is not null and position between 1 and 5
group by state
order by restro_count desc
limit 10;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,state,restro_count
0,Texas,1930
1,Virginia,1177
2,Wisconsin,1059
3,Washington,929
4,West Virginia,428
5,Utah,371
6,Alabama,191
7,Vermont,177
8,Wyoming,134
9,None,38


#### To know:
- Top states with restaurants ranked between position 1 and 5.

#### Insight:
- `Texas`, `Virginia` and `Wisconsin` have the most top-ranked restaurants in this range.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>8. Restaurants Rank by score within each State.</b>
</div>

In [93]:
df = pd.read_sql("""


with ranked_restaurants as (
    select 
        state, 
        name as restro_name,
        score,
        rank() over (partition by state order by score desc) as state_rank
    from restro
    where score is not null and state is not null
    )
select * from ranked_restaurants
where state_rank = 1
order by score desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,state,restro_name,score,state_rank
0,Alabama,"Honey Baked Ham (7001 Crestwood Blvd, Ste 114)",5.0,1
1,Alabama,Hunt Brothers Pizza,5.0,1
2,Alabama,Cookie Dough Magic,5.0,1
3,Alabama,Jeni's Splendid Ice Cream (Pepper Place),5.0,1
4,Alabama,Chick-fil-A (2502 Highway 31 S),5.0,1
...,...,...,...,...
925,Tennessee,"Chicken Salad Chick (0156 - Bristol, TN)",4.8,1
926,Idaho,Starbucks (1930 West Pullman Road),4.6,1
927,Idaho,Mad Greek,4.6,1
928,Nebraska,Sushi Kitchen,4.6,1


- Why:

     To find the best-rated restaurant(s) in each state.

- Insight:

     Many states have more than one top-rated restaurant with the same highest score.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>9. Creating a view showing the number of restaurants per State, including restro count and average rating.</b>
</div>

In [ ]:
df = pd.read_sql("""

create or replace view state_summery as 
select 
    state, 
    count(*) as restro_count,
    round(avg(score), 2) as avg_score
from restro
where score is not null
group by state;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

#### To know:
- How many restaurants are in each state and their average rating.

#### Insight:
- Some States have `1000+ restaurants` with good average ratings. `Nebraska` appear only once.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>10. List of restaurants whose category list contains more than 10 cuisine type.</b>
</div>

In [95]:
df = pd.read_sql("""

with category_counts as (
    select 
        name as restro,
        category,
        length(category) - length(replace(category, ',', '')) + 1 as cuisine_count
    from restro
    where category is not null
)
select * from category_counts
where cuisine_count > 10;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restro,category,cuisine_count
0,Hickory Tavern (Brookwood Village),"American, New American, Wings, Burgers, Burger...",18
1,Applebee's (5 Points West),"American, Bar Food, BBQ, burger, Chicken Strip...",15
2,Applebee's (Fultondale),"American, Bar Food, BBQ, burger, Chicken Strip...",15
3,Applebee's (Gadsden),"American, Bar Food, BBQ, burger, Chicken Strip...",15
4,FireBurger (191 Main Street),"American, Traditional American, Burgers, Burge...",12
...,...,...,...
540,Taste of Successs,"Caribbean, Seafood, Drinks, Allergy Friendly, ...",16
541,Burgers &amp; Beyond,"Burgers, American, Salads, Sandwiches, Chicken...",11
542,Blue Goose Cantina - McKinney,"Mexican, Latin American, New Mexican, Alcohol,...",23
543,China Star,"Chinese, Asian, Asian Fusion, BBQ, Rolls, Wing...",12


#### To know:
- To find restaurants that offer a large variety of cuisines.

#### Insight:
- More than `500` restaurants serve over 10 types of cuisine, meaning they don’t stick to just one type like only Pizza or only Chinese. They try to offer more options to attract more people.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>11. Restaurants where the difference between `position` and `average position` in that state is more than 100.</b>
</div>

In [96]:
df = pd.read_sql("""

select 
    r.id,
    r.name AS restro,
    r.state,
    r.position,
    c.avg_position,
    round(c.avg_position - r.position, 2) AS diff
from restro as r
join (
    select state, avg(position) as avg_position
    from restro
    where position is not null
    group by state
) as c on r.state = c.state
 where r.position is not null
 and abs(c.avg_position - r.position) > 100
 order by diff desc;
 
""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,id,restro,state,position,avg_position,diff
0,10946,The Mandarin House,Oregon,1,134.9641,133.96
1,11252,Amico Chef,Oregon,1,134.9641,133.96
2,11259,Hoda's Lebanese Restaurant,Oregon,1,134.9641,133.96
3,11512,E’Njoni Cafe,Oregon,1,134.9641,133.96
4,9948,Von Ebert Brewing Pizzeria and Taproom (Cascade),Oregon,2,134.9641,132.96
...,...,...,...,...,...,...
8977,2025,JJ Fish and Chicken (3213 W Villard Ave),Wisconsin,297,45.6418,-251.36
8978,2024,"Fresh, Fast &amp; Delicious",Wisconsin,298,45.6418,-252.36
8979,1876,Taqueria El Paso,Wisconsin,299,45.6418,-253.36
8980,2022,Rojo Blanco fast food,Wisconsin,299,45.6418,-253.36


#### To know:
- To find restaurants whose position is very different from the average position in their state — whether much better or worse.

#### Insight:
- Some restaurants are doing much worse than others in states that usually have high average positions, while some are doing much better even in states with poor averages.
This shows a performance gap — helping identify outliers, or areas need improvements.

<div style="background-color:#e2f0d9; padding:15px; border-radius:15px; color:#003366; font-family:Arial; font-size:18px;">

<span style="font-size:30px; font-weight:bold;">3. analysis on joined `restaurant` + `restaurant_menu`</span><br><br>

This section explores combined data from both the restaurant and menu tables to analyze restaurant-level pricing, offerings, and performance.
</div>

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>1. Restaurants with max., min. and average prices. </b>

In [97]:
df = pd.read_sql("""

select 
    r.name,
    min(m.price) as Min_price, 
    max(m.price) as Max_price, 
    round(avg(m.price), 2) as Avg_price
from restro as r
join menu as m on r.id = m.restaurant_id
where m.price is not null
group by r.name
order by Avg_price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,name,Min_price,Max_price,Avg_price
0,Lucky Moo Foods - Wattala,150.0,180.0,170.00
1,The Healthy Hub,140.0,150.0,145.00
2,IHOP (Reynosa),45.0,303.0,140.49
3,Le Labo (1011 South Congress Ave ),22.0,289.0,130.24
4,Le Labo (65 Highland Park Village ),16.0,289.0,127.21
...,...,...,...,...
60287,Dunkin' (2203 Spring Stuebner Rd),0.0,0.0,0.00
60288,"Baskin-Robbins (1320 Zaragoza Road, Ste 122)",0.0,0.0,0.00
60289,Dunkin (7939 Pat Booker Rd),0.0,0.0,0.00
60290,Dunkin' (2125 Main St),0.0,0.0,0.00


- Why?

    To see how cheap or expensive each restaurant is by checking their lowest, highest, and average item prices.

- Insight:

    Some restaurants have a wide price range, while others have similar prices — this helps us know which places are cheap or costly.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>2. Count of items availale in each restaurant.</b>
</div>

In [98]:
df = pd.read_sql("""

select 
    m.restaurant_id, 
    r.name as restro, 
    count(distinct(m.name)) as menu_item_count
from restro as r
left join menu as m on m.restaurant_id = r.id
where m.name is not null
group by m.restaurant_id, restro
order by menu_item_count desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restaurant_id,restro,menu_item_count
0,47639,Worth The Pour,2278
1,2811,Clark Gas Station - 1831 West National Ave. (P...,2075
2,26674,Exotic Wine Spirits,1895
3,28738,Exotic Wine and Liquors (2300 Washington Pl Ne),1894
4,56144,Brand Liquors,1471
...,...,...,...
63105,23938,XTREMEROLLS,1
63106,25555,Royalty Caribbean and Soul Food,1
63107,25958,The Skinny Dip Frozen Yogurt Bar,1
63108,27229,The Ice Cream Shop,1


- Why?

    To count how many menu items each restaurant offers, showing their menu size.

- Insight:

    Some restaurants (like liquor shops) have 2000+ items, showing broad inventory, while others with just 1 item may indicate limited offerings

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>3. Restaurants that offer more then 100 items in their menu.</b>
</div>

In [99]:
df = pd.read_sql("""

select 
    m.restaurant_id, 
    r.name as restro, 
    count(distinct(m.name)) as menu_item_count
from restro as r
left join menu as m on m.restaurant_id = r.id
where m.name is not null
group by m.restaurant_id, restro
having menu_item_count > 100
order by menu_item_count desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restaurant_id,restro,menu_item_count
0,47639,Worth The Pour,2278
1,2811,Clark Gas Station - 1831 West National Ave. (P...,2075
2,26674,Exotic Wine Spirits,1895
3,28738,Exotic Wine and Liquors (2300 Washington Pl Ne),1894
4,56144,Brand Liquors,1471
...,...,...,...
13766,29409,Denny's (361 South Main Street),101
13767,29513,Birdseye Diner,101
13768,31189,Buffalo Wild Wings (221 S River Rd),101
13769,31195,Jalapeños Authentic Mexican Food (Bluff St),101


- Why:

     To find restaurants with more than 100 items on their menu.

- Insight:

     Mostly liquor or convenience stores show up, meaning they sell a wide range of products, not just food.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>4. Min, Max, Average menu item price per restaurant.</b>
</div>

In [100]:
df = pd.read_sql("""

select 
    m.restaurant_id, 
    r.name as restro, 
    min(price) as min_price,
    max(price) as max_price,
    avg(price) as avg_menu_price
from restro as r
left join menu as m on m.restaurant_id = r.id
where price is not null 
group by m.restaurant_id, restro
having avg_menu_price != 0
order by avg_menu_price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restaurant_id,restro,min_price,max_price,avg_menu_price
0,54680,Lucky Moo Foods - Wattala,150.0,180.00,170.000000
1,58177,The Healthy Hub,140.0,150.00,145.000000
2,57202,IHOP (Reynosa),45.0,303.00,140.488889
3,19449,Fish Market Restaurant,18.0,275.00,135.853659
4,40577,Le Labo (1011 South Congress Ave ),22.0,289.00,130.240196
...,...,...,...,...,...
62561,5637,Casey's (1871 E Inman Pkwy),0.0,3.99,0.025352
62562,5570,Casey's (602 W North St),0.0,3.99,0.025208
62563,1694,Casey's (2322 Jackson St),0.0,3.79,0.019915
62564,3830,Casey's (1306 4 Mile Rd),0.0,3.79,0.018982


- Why:

     To understand each restaurant’s pricing range and average item cost.

- Insight:

     Some restaurants have very high average prices (premium dining), while others like Casey’s have extremely low prices, likely due to low-cost

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>5. States along with the average item price of its restaurants.</b>
</div>

In [101]:
df = pd.read_sql("""

select 
    State, 
    min(price) as min_price,
    max(price) as max_price,
    avg(price) as avg_menu_price
from restro as r
left join menu as m on m.restaurant_id = r.id
where price is not null 
group by State
having avg_menu_price != 0
order by avg_menu_price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,State,min_price,max_price,avg_menu_price
0,None,0.00,429.99,12.650385
1,Maryland,0.00,1099.99,11.824522
2,Illinois,0.00,359.99,11.681638
3,Idaho,0.00,249.99,11.442769
4,Oregon,0.00,289.00,11.050457
5,Washington,-7.05,1099.99,10.953601
6,Nebraska,0.85,41.20,10.931011
7,Missouri,1.89,49.99,10.896593
8,Texas,-2.50,1395.00,10.690294
9,Tennessee,0.00,250.00,10.600128


- Why:

     To compare how food prices vary across different states.

- Insight:

     Some states have higher average prices, showing more expensive dining, while others are more affordable.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>6. Menu Items Priced Above Their Category Average Using CTE.</b>
</div>

In [102]:
df = pd.read_sql("""

with cat_averages as (
    select 
        category, 
        round(avg(price), 2) as avg_category_prices
    from menu
    where price is not null
    group by category
) 
select 
    m.category, 
    m.name, 
    m.price, 
    ca.avg_category_prices from menu as m 
join cat_averages as ca on m.category = ca.category
where m.price > ca.avg_category_prices
order by m.category, m.price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,category,name,price,avg_category_prices
0,\nBeverages,Ramune,4.00,2.77
1,\nBeverages,Caipoco,3.00,2.77
2,\nBeverages,Iced Oolong Tea,3.00,2.77
3,\nBeverages,Iced Green Tea,3.00,2.77
4,\nRamen,Spicy Miso,12.95,9.71
...,...,...,...,...
2139516,🧊Beverages🧊,Smart Water (33.8 oz),3.99,2.78
2139517,🧊Beverages🧊,Starbucks Frappuccino,3.99,2.78
2139518,🧊Beverages🧊,Redbull,3.99,2.78
2139519,🧊Beverages🧊,Monster,3.99,2.78


- Why:

     To find items that are priced higher than the average in their category.

- Insight:

  These items stand out as premium or possibly overpriced compared to similar options.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>7. Restaurants with at least one item over $100.</b>
</div>

In [103]:
df = pd.read_sql("""

SELECT r.id, r.name AS restaurant_name, price
FROM restro as r
JOIN menu as m ON r.id = m.restaurant_id
WHERE m.price > 100
order by m.price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,id,restaurant_name,price
0,54106,Fire Ass Thai,1395.00
1,38155,buybuy BABY (4648 SW Loop 820),1099.99
2,38155,buybuy BABY (4648 SW Loop 820),1099.99
3,38155,buybuy BABY (4648 SW Loop 820),1099.99
4,23182,"buybuy BABY (24670 Dulles Landing Dr,Unit 130)",1099.99
...,...,...,...
12366,47538,Shell Shack - Plano,100.75
12367,50618,Big 6 Bar B Que,100.49
12368,47639,Worth The Pour,100.37
12369,47639,Worth The Pour,100.37


- Why:

     To find restaurants with items over $100.

- Insight:

     Fire Ass Thai sells an item for $1395, and several buybuy BABY locations list products at 1099.99—showing some restaurants offer very high-priced items.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>8. Restaurant names along with the number of high-priced items (above $200) they offer.</b>
</div>

In [104]:
df = pd.read_sql("""

SELECT r.name AS restro_name, count(*) as high_price_item_count  
FROM restro as r
JOIN menu as m ON r.id = m.restaurant_id
WHERE m.price is not null and m.price > 100
group by restro_name
order by high_price_item_count desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restro_name,high_price_item_count
0,Le Labo (1308 W Burnside St ),102
1,Le Labo (65 Highland Park Village ),100
2,Le Labo (1011 South Congress Ave ),99
3,Le Labo (921 E Pine St ),99
4,Le Labo (1665 Westheimer Rd Ste B-1 ),99
...,...,...
3001,Walgreens (156 Fm 518 Rd),1
3002,Raising Cane's Chicken Fingers (4760 W Univers...,1
3003,Pasqually's Pizza &amp; Wings P943 (401 W Loui...,1
3004,StoneAge Korean Grill and Bar,1


- Why:

     to count how many expensive (above $200) items each restaurant sells.

- Insight:

     Le Labo locations offer the most high-priced items (up to 102), showing they specialize in premium or luxury products, while many others offer just one.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>9. Restaurant names along with the number of low-priced items (under $5) they offer.</b>
</div>

In [105]:
df = pd.read_sql("""

SELECT r.name AS restro_name, count(*) as low_price_item_count 
FROM restro as r
JOIN menu as m ON r.id = m.restaurant_id
WHERE m.price is not null and m.price < 5
group by restro_name
order by low_price_item_count desc
limit 10;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restro_name,low_price_item_count
0,Everyday Needs by Gopuff,4170
1,Clark Gas Station - 1831 West National Ave. (P...,1751
2,Pollo Regio,1595
3,Mr. Express Corner Store,1196
4,Panera Bread,1090
5,Hopin Market,888
6,Godiva Chocolatier (1961 Chain Bridge Rd),812
7,Indiana market,774
8,Little Chief Mini Mart,707
9,IndeeFresh,655


- Why:

     To find how many cheap items (under $5) each restaurant offers.

- Insight:

     Stores like Everyday Needs by Gopuff and Clark Gas Station offer thousands of low-price items, showing they focus on budget-friendly or retail products.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>10. Restaurants whose most expensive item is over $200.</b>
</div>

In [106]:
df = pd.read_sql("""

SELECT r.name AS restro_name, sub.max_price
FROM restro AS r  
JOIN (
    SELECT restaurant_id, MAX(price) AS max_price  
    FROM menu  
    WHERE price IS NOT NULL  
    GROUP BY restaurant_id
) AS sub ON r.id = sub.restaurant_id  
WHERE sub.max_price > 200
order by sub.max_price desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restro_name,max_price
0,Fire Ass Thai,1395.00
1,buybuy BABY (4351 Creekside Ave.),1099.99
2,buybuy BABY (665 Main Street),1099.99
3,buybuy BABY (17686 Southcenter Parkway),1099.99
4,buybuy BABY (2700 Potomac Mills Circle Suite 100),1099.99
...,...,...
832,Freshii (Gaston Ave &amp; N Washington Ave),203.99
833,Happy Chicks (Burnet Rd),203.55
834,Yardbird Southern Table &amp; Bar (DC),201.00
835,"Joe's Seafood, Prime Steak &amp; Stone Crab 15...",200.95


- Why:

     To find restaurants where the most expensive item costs over $200.

- Insight:

     Places like Fire Ass Thai and buybuy BABY offer very high-priced items, showing luxury offerings or non-food products.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>11. Restaurants where the price gap (max-min) is more than $100.</b>
</div>

In [107]:

df = pd.read_sql("""

SELECT r.id, 
    r.name AS restro_name, 
    MAX(m.price) as max_price, 
    MIN(m.price) as min_price,
    MAX(m.price) - MIN(m.price) AS price_gap  
FROM restro AS r  
JOIN menu AS m ON r.id = m.restaurant_id  
WHERE m.price IS NOT NULL  
GROUP BY r.id, r.name  
HAVING price_gap > 100
order by price_gap desc;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,id,restro_name,max_price,min_price,price_gap
0,54106,Fire Ass Thai,1395.00,2.00,1393.00
1,2633,buybuy BABY (665 Main Street),1099.99,0.48,1099.51
2,51770,buybuy BABY (12710 Fountain Lake Circle),1099.99,0.99,1099.00
3,18263,buybuy BABY (2700 Potomac Mills Circle Suite 100),1099.99,0.99,1099.00
4,485,buybuy BABY (4351 Creekside Ave.),1099.99,1.29,1098.70
...,...,...,...,...,...
2919,38566,Chicken Wing Co. (206 N Oak St),101.19,1.10,100.09
2920,18260,Chicken Wing Co. (5860 Mapledale Plaza),101.19,1.10,100.09
2921,18275,Wingman Co. (5860 Mapledale Plaza),101.19,1.10,100.09
2922,38990,The Wing Bar (206 N Oak St),101.19,1.10,100.09


- Why?

  To find restaurants that have a big difference between their cheapest and most expensive menu items.

- insight :
  The difference between the highest and lowest menu prices shows how much the restaurant’s prices vary. A big difference means the restaurant offers both cheap and expensive items. A small difference means most items are priced similarly. This helps us understand the type of menu and customers they target.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>12. Top 5 restaurants with the highest average menu item price.</b>
</div>

In [108]:
df = pd.read_sql("""

SELECT 
    r.id,
    r.name AS restro_name,
    AVG(m.price) AS avg_price
FROM restro AS r
JOIN menu AS m ON r.id = m.restaurant_id
WHERE m.price IS NOT NULL
GROUP BY r.id, r.name
ORDER BY avg_price DESC
LIMIT 10;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,id,restro_name,avg_price
0,54680,Lucky Moo Foods - Wattala,170.000000
1,58177,The Healthy Hub,145.000000
2,57202,IHOP (Reynosa),140.488889
3,19449,Fish Market Restaurant,135.853659
4,40577,Le Labo (1011 South Congress Ave ),130.240196
5,60251,Le Labo (65 Highland Park Village ),127.209302
6,11683,Le Labo (1308 W Burnside St ),127.115741
7,52968,Le Labo (1665 Westheimer Rd Ste B-1 ),126.069444
8,20804,Le Labo (3005 M Street NW ),125.449074
9,13322,Le Labo (921 E Pine St ),125.179724


- Why:

     To identify the top 5 restaurants with the highest average menu prices.

- Insight:

     Lucky Moo Foods, The Healthy Hub, and IHOP (Reynosa) lead with average prices over $140, showing they target premium or upscale markets.

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:17px;">
<b>13. Total (sum) of all menu item prices for each restaurant.</b>
</div>

In [109]:
df = pd.read_sql("""

SELECT
    r.id AS restro_id,
    r.name AS restro_name,
    SUM(m.price) AS total_menu_price
FROM restro AS r
JOIN menu AS m ON r.id = m.restaurant_id
WHERE m.price IS NOT NULL
GROUP BY r.id, r.name
ORDER BY total_menu_price DESC;

""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

,restro_id,restro_name,total_menu_price
0,47639,Worth The Pour,69787.74
1,51780,Bargain Liquor,51776.07
2,56144,Brand Liquors,50894.58
3,19965,Hamilton Liquors (5205 Georgia Ave NW),48132.90
4,28738,Exotic Wine and Liquors (2300 Washington Pl Ne),47198.49
...,...,...,...
63105,50489,Dunkin' (6045 Montana Ave),0.00
63106,52487,Baskin Robbins (14501 Memorial Dr),0.00
63107,53629,Dunkin' (2222 Rayford Rd),0.00
63108,54860,Dunkin' (4130 Fairmont Pkwy),0.00


- Why:

     To know how much money each restaurant could make if all items were sold once.

- Insight:

     Worth The Pour has the highest total menu value at `$69,787.74 followed by Bargain Liquor and Brand Liquors, indicating a large and/or high-priced menu. Some restaurants show $0`, meaning they might have missing or unpriced menu items.